# Practica Final UD2 SAA

## Parte 1: Análisis de Ventas de una Tienda

In [2]:
import pandas as pd
import numpy as np

datos_ventas = {
    'Venta_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10], # Hay una fila duplicada
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02', '2023-01-03', '2023-01-08', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-11'],
    'Producto_ID': [101, 102, 101, 103, 104, 101, 102, 103, 102, 101, 101],
    'Vendedor_ID': [1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1],
    'Cantidad': [20, 5, 25, 7, 10, 18, 6, 8, np.nan, 15, 15], # Hay un valor nulo
    'Precio_str': ['$ 5.00', '€ 20.50', '$ 5.50', '$ 8.00', '€ 3.00', '$ 5.25', '$ 21.00', '$ 8.50', '€ 22.00', '$ 5.00', '$ 5.00'],
    'Sucursal': ['Norte', '  Sur', 'norte', 'Norte', 'Sur', 'Norte', 'Sur', 'Norte', 'Sur', '  norte  ', '  norte  '] # Nombres inconsistentes
}

datos_productos = {
    'Producto_ID': [101, 102, 103, 104],
    'Nombre_Producto': ['Café', 'Tarta', 'Pastel', 'Té'],
    'Categoría': ['Bebida', 'Postre', 'Postre', 'Bebida']
}

datos_vendedores = {
    'Vendedor_ID': [1, 2],
    'Nombre_Vendedor': ['Ana', 'Luis']
}

### Limpieza de datos

In [23]:
df_ventas = pd.DataFrame(datos_ventas)
df_productos = pd.DataFrame(datos_productos)
df_vendedores = pd.DataFrame(datos_vendedores)

df_ventas = df_ventas.drop_duplicates()

df_ventas = df_ventas.dropna()

df_ventas['Sucursal'] = df_ventas['Sucursal'].str.strip().str.lower()

df_ventas['Precio_str'] = df_ventas['Precio_str'].str.replace('[^0-9.]', '', regex=True).astype(float)

df_final = df_ventas.merge(df_productos, on='Producto_ID', how='left')
df_final = df_final.merge(df_vendedores, on='Vendedor_ID', how='left')

df_final['Ingresos_Totales'] = df_final['Cantidad'] * df_final['Precio_str']

df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])
df_final = df_final.set_index('Fecha')

print(df_final.head())

            Venta_ID  Producto_ID  Vendedor_ID  Cantidad  Precio_str Sucursal  \
Fecha                                                                           
2023-01-01         1          101            1      20.0         5.0    norte   
2023-01-01         2          102            2       5.0        20.5      sur   
2023-01-02         3          101            1      25.0         5.5    norte   
2023-01-02         4          103            1       7.0         8.0    norte   
2023-01-03         5          104            2      10.0         3.0      sur   

           Nombre_Producto Categoría Nombre_Vendedor  Ingresos_Totales  
Fecha                                                                   
2023-01-01            Café    Bebida             Ana             100.0  
2023-01-01           Tarta    Postre            Luis             102.5  
2023-01-02            Café    Bebida             Ana             137.5  
2023-01-02          Pastel    Postre             Ana              5

### Análisis Exploratorio y Creación de Variables

In [14]:
ingresos_categoria_vendedor = df_final.groupby(['Categoría', 'Nombre_Vendedor'])['Ingresos_Totales'].sum()
print("Ingresos totales por Categoría y por Vendedor:")
print(ingresos_categoria_vendedor, '\n')

cantidad_categoria_sucursal = df_final.pivot_table(values='Cantidad', index='Categoría', columns='Sucursal', aggfunc='sum')
print("Cantidad total de productos vendidos por Categoría y Sucursal:")
print(cantidad_categoria_sucursal, '\n')

df_final['Tipo_Venta'] = np.where(df_final['Ingresos_Totales'] > 50, 'Grande', 'Normal')

producto_estrella = (
    df_final.groupby(['Categoría', 'Nombre_Producto'])['Ingresos_Totales']
    .sum()
    .reset_index()
)
producto_estrella = producto_estrella.loc[
    producto_estrella.groupby('Categoría')['Ingresos_Totales'].idxmax()
]
print("Producto Estrella por Categoría:")
print(producto_estrella.reset_index(drop=True), '\n')

df_final['Pct_Ingreso_Categoria'] = (
    df_final['Ingresos_Totales'] / df_final.groupby('Categoría')['Ingresos_Totales'].transform('sum') * 100
)

print("DataFrame final con nuevas columnas:")
print(df_final[['Nombre_Producto', 'Categoría', 'Nombre_Vendedor', 'Ingresos_Totales', 'Tipo_Venta', 'Pct_Ingreso_Categoria']].head())

Ingresos totales por Categoría y por Vendedor:
Categoría  Nombre_Vendedor
Bebida     Ana                407.0
           Luis                30.0
Postre     Ana                124.0
           Luis               228.5
Name: Ingresos_Totales, dtype: float64 

Cantidad total de productos vendidos por Categoría y Sucursal:
Sucursal   norte   sur
Categoría             
Bebida      78.0  10.0
Postre      15.0  11.0 

Producto Estrella por Categoría:
  Categoría Nombre_Producto  Ingresos_Totales
0    Bebida            Café             407.0
1    Postre           Tarta             228.5 

DataFrame final con nuevas columnas:
           Nombre_Producto Categoría Nombre_Vendedor  Ingresos_Totales  \
Fecha                                                                    
2023-01-01            Café    Bebida             Ana             100.0   
2023-01-01           Tarta    Postre            Luis             102.5   
2023-01-02            Café    Bebida             Ana             137.5   
2023

### Análisis Temporal

In [ ]:
ingresos_diarios = df_final['Ingresos_Totales'].resample('D').sum()
ingresos_semanales = df_final['Ingresos_Totales'].resample('W').sum()

#Esto se ha puesto con el reset_index para que no aparezca el texto de pandas de la operación que se ha hecho sobre la serie.
print("Ingresos Totales Diarios:")
print(ingresos_diarios.reset_index(name='Ingresos_Totales'), '\n')

print("Ingresos Totales Semanales:")
print(ingresos_semanales.reset_index(name='Ingresos_Totales'), '\n')

df_final['Dia_Semana'] = df_final.index.day_name()

ingresos_por_dia = (
    df_final.groupby('Dia_Semana')['Ingresos_Totales']
    .mean()
    .sort_values(ascending=False)
)

print("Promedio de Ingresos por Día de la Semana:")
print(ingresos_por_dia.reset_index(name='Promedio_Ingresos'), '\n')

dia_top = ingresos_por_dia.idxmax()
valor_top = ingresos_por_dia.max()

print(f"El día con mayor promedio de ingresos es **{dia_top}** con ${valor_top:.2f} en promedio.")

Ingresos Totales Diarios:
        Fecha  Ingresos_Totales
0  2023-01-01             202.5
1  2023-01-02             193.5
2  2023-01-03              30.0
3  2023-01-04               0.0
4  2023-01-05               0.0
5  2023-01-06               0.0
6  2023-01-07               0.0
7  2023-01-08             220.5
8  2023-01-09              68.0
9  2023-01-10               0.0
10 2023-01-11              75.0 

Ingresos Totales Semanales:
       Fecha  Ingresos_Totales
0 2023-01-01             202.5
1 2023-01-08             444.0
2 2023-01-15             143.0 

Promedio de Ingresos por Día de la Semana:
  Dia_Semana  Promedio_Ingresos
0     Sunday         105.750000
1     Monday          87.166667
2  Wednesday          75.000000
3    Tuesday          30.000000 

El día con mayor promedio de ingresos es **Sunday** con $105.75 en promedio.


## Parte 2: Análisis Exploratorio de Datos de Inmuebles

In [22]:
datos_inmuebles = {
    'Referencia': ['VLC-01', 'MAD-01', 'BCN-01', 'VLC-02', 'MAD-02', 'BCN-02', 'VLC-03', 'MAD-03', 'BCN-03', 'VLC-04'],
    'Ciudad': ['Valencia', 'Madrid', 'Barcelona', 'Valencia', 'Madrid', 'Barcelona', 'Valencia', 'Madrid', 'Barcelona', 'Valencia'],
    'Habitaciones': [3, 4, 2, 4, 5, 3, 2, 3, 4, 3],
    'Baños': [2, 2, 1, 3, 3, 2, 1, 2, 2, 2],
    'Superficie': [90, 120, 65, 110, 150, 85, 55, 95, 115, 100],
    'Precio': [210000, 450000, 320000, 280000, 600000, 410000, 180000, 390000, 550000, 250000],
    'Tipo_Propiedad': ['Piso', 'Piso', 'Apartamento', 'Piso', 'Chalet', 'Piso', 'Apartamento', 'Piso', 'Piso', 'Piso'],
    'Extras': ['Garaje', 'Piscina', np.nan, 'Garaje;Terraza', 'Piscina;Jardín', 'Terraza', np.nan, 'Garaje', 'Garaje;Piscina', 'Terraza']
}

### Preguntas de Negocio a Resolver

#### Preparación de Datos

In [ ]:
df_inmuebles = pd.DataFrame(datos_inmuebles)

#Hemos decidido aplicar un texto simple como "Sin extras" para que se vea mas claramente,
df_inmuebles['Extras'] = df_inmuebles['Extras'].fillna('Sin extras')

#Aqui hemos decidido hacer un slpit para que sea una lista para mejor visualización y posible posterior uso con el método .explode()
df_inmuebles['Extras_lista'] = df_inmuebles['Extras'].str.split(';')

df_inmuebles['Precio_m2'] = df_inmuebles['Precio'] / df_inmuebles['Superficie']

print(df_inmuebles[['Referencia', 'Ciudad', 'Superficie', 'Precio', 'Precio_m2', 'Extras_lista']])

  Referencia     Ciudad  Superficie  Precio    Precio_m2       Extras_lista
0     VLC-01   Valencia          90  210000  2333.333333           [Garaje]
1     MAD-01     Madrid         120  450000  3750.000000          [Piscina]
2     BCN-01  Barcelona          65  320000  4923.076923       [Sin extras]
3     VLC-02   Valencia         110  280000  2545.454545  [Garaje, Terraza]
4     MAD-02     Madrid         150  600000  4000.000000  [Piscina, Jardín]
5     BCN-02  Barcelona          85  410000  4823.529412          [Terraza]
6     VLC-03   Valencia          55  180000  3272.727273       [Sin extras]
7     MAD-03     Madrid          95  390000  4105.263158           [Garaje]
8     BCN-03  Barcelona         115  550000  4782.608696  [Garaje, Piscina]
9     VLC-04   Valencia         100  250000  2500.000000          [Terraza]


#### Análisis del Mercado

In [42]:
df_inmuebles.groupby('Ciudad')[['Precio','Superficie']].describe()

Precio                                                              \
           count           mean            std       min       25%       50%   
Ciudad                                                                         
Barcelona    3.0  426666.666667  115902.257671  320000.0  365000.0  410000.0   
Madrid       3.0  480000.000000  108166.538264  390000.0  420000.0  450000.0   
Valencia     4.0  230000.000000   43969.686528  180000.0  202500.0  230000.0   

                              Superficie                                       \
                75%       max      count        mean        std   min     25%   
Ciudad                                                                          
Barcelona  480000.0  550000.0        3.0   88.333333  25.166115  65.0   75.00   
Madrid     525000.0  600000.0        3.0  121.666667  27.537853  95.0  107.50   
Valencia   257500.0  280000.0        4.0   88.750000  23.935678  55.0   81.25   

                                
             50%    75%    max  
Ciudad                          
Barcelona   85.0  100.0  115.0  
Madrid     120.0  135.0  150.0  
Valencia    95.0  102.5  110.0

La ciudad mas cara parece ser Madrid, ya que tiene mas precio medio, siendo Barcelona la segunda y Valencia con casi la mitad.

In [43]:
for ciudad, datos in df_inmuebles.groupby('Ciudad'):
    print(f"\nCiudad: {ciudad}")
    print(datos[['Habitaciones', 'Baños', 'Precio']].corr())


Ciudad: Barcelona
              Habitaciones     Baños    Precio
Habitaciones      1.000000  0.866025  0.992215
Baños             0.866025  1.000000  0.797017
Precio            0.992215  0.797017  1.000000

Ciudad: Madrid
              Habitaciones     Baños    Precio
Habitaciones      1.000000  0.866025  0.970725
Baños             0.866025  1.000000  0.960769
Precio            0.970725  0.960769  1.000000

Ciudad: Valencia
              Habitaciones     Baños    Precio
Habitaciones      1.000000  1.000000  0.928477
Baños             1.000000  1.000000  0.928477
Precio            0.928477  0.928477  1.000000


En Barcelona parece que a mas habitaciones mas caro es, mientras que los baños tienen menos correlación, en Madrid no ocurre tanto pero aun hay algo de disparidad, finalmente en Valencia parecen tener el mismo valor las habitaciones y los baños.

In [44]:
tipo_stats = df_inmuebles.groupby('Tipo_Propiedad')[['Precio', 'Superficie']].mean()
tipo_stats['Precio_m2'] = tipo_stats['Precio'] / tipo_stats['Superficie']
print(tipo_stats.sort_values('Precio', ascending=False))

# Tipo más caro y más grande
tipo_mas_caro = tipo_stats['Precio'].idxmax()
tipo_mas_grande = tipo_stats['Superficie'].idxmax()

print(f"\nTipo más caro: {tipo_mas_caro}")
print(f"Tipo con mayor superficie media: {tipo_mas_grande}")

                       Precio  Superficie    Precio_m2
Tipo_Propiedad                                        
Chalet          600000.000000  150.000000  4000.000000
Piso            362857.142857  102.142857  3552.447552
Apartamento     250000.000000   60.000000  4166.666667

Tipo más caro: Chalet
Tipo con mayor superficie media: Chalet


Ciertamente los chalets son con diferencia las propiedades mas caras y con mayor superficioe, los pisos despúes aunque son los mas baratos por metro cuadrado mientras que los apartamentos son mas caros en proporcion pero mas baratos que todos en general.

In [45]:
df_extras = df_inmuebles.explode('Extras_lista')

tiene_piscina = df_extras[df_extras['Extras_lista'].str.contains('Piscina', na=False)]
tiene_garaje = df_extras[df_extras['Extras_lista'].str.contains('Garaje', na=False)]

porcentaje_piscina = (
    tiene_piscina.groupby('Ciudad')['Referencia'].nunique() / 
    df_inmuebles.groupby('Ciudad')['Referencia'].nunique() * 100
)

porcentaje_garaje = (
    tiene_garaje.groupby('Ciudad')['Referencia'].nunique() / 
    df_inmuebles.groupby('Ciudad')['Referencia'].nunique() * 100
)

porcentaje_piscina = porcentaje_piscina.fillna(0).rename(None)
porcentaje_garaje = porcentaje_garaje.fillna(0).rename(None)

print("Porcentaje con piscina por ciudad:")
print(porcentaje_piscina.reset_index(name='Porcentaje_Piscina'), '\n')

print("Porcentaje con garaje por ciudad:")
print(porcentaje_garaje.reset_index(name='Porcentaje_Garaje'), '\n')


Porcentaje con piscina por ciudad:
      Ciudad  Porcentaje_Piscina
0  Barcelona           33.333333
1     Madrid           66.666667
2   Valencia            0.000000 

Porcentaje con garaje por ciudad:
      Ciudad  Porcentaje_Garaje
0  Barcelona          33.333333
1     Madrid          33.333333
2   Valencia          50.000000 



De los edificios, Madrid tiene 2/3 de piscinas en sus edificios mientras que Barcelona solo tiene 1/3, con Valencia no teniendo ninguna piscina

Con los Garajes, la mitad de los edificios de Valencia tienen garaje mientras que tanto Barcelona como Madrid solo tienen un tercio de sus viviendas